In [5]:
import pandas as pd
import numpy as np

data_dir = r"C:\Users\hp\Desktop\facial-expression-recognition"

df=pd.read_csv(data_dir+"/train.csv")

In [6]:

emotion1 = ('anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise', 'neutral')


In [7]:
exclude_classes1 = [0,1, 2,4, 5,6]

# Delete rows based on the condition
df_happy = df[~df['emotion'].isin(exclude_classes1)]

In [8]:
exclude_classes2 = [0,1, 2,3, 5,6]

# Delete rows based on the condition
df_sad =  df[~df['emotion'].isin(exclude_classes2)]

In [9]:
df1_subset = df_sad.head(40)
df2_subset = df_happy.head(40)

concatenated_df = pd.concat([df1_subset, df2_subset])

concatenated_df.reset_index(drop=True, inplace=True)


In [10]:
emotion = ('happiness', 'sadness')


In [11]:
X = concatenated_df['pixels'].apply(lambda x: np.array(x.split()).reshape(48*48,).astype('float32'))
X = np.stack(X, axis=0)
X.shape

(80, 2304)

In [12]:
y = concatenated_df['emotion']
y = np.stack(y, axis=0)

y.shape

(80,)

In [13]:
for i in range(len(y)):
    if y[i] == 3:
        y[i] = 0
        
    if y[i] == 4:
        y[i] = 1


print(y)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]


In [14]:
X1 = X/255.

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size=0.20, random_state=42)

In [16]:
import random as rd
import numpy as np

class FCM:
    def __init__(self,X,c,m,m_iter):
        self.m = m
        self.c = c
        self.p = X.shape[1]
        self.m_iter = m_iter
        
        self.V = np.array([ [ rd.uniform(0,1) for j in range(self.p)] for i in range(self.c)])
        self.Etqt = []

    def calculerU(self, X):
        alpha = -1./(self.m-1.)
        term1 = [ [ pow(sum(np.square(X[k]-self.V[i])), alpha) for k in range(X.shape[0]) ] for i in range(self.c)]
        term2 = [ sum([pow(sum(np.square(X[k]-self.V[i])), alpha) for i in range(self.c)]) for k in range(X.shape[0])]
        self.U = [ [ term1[i][k]/term2[k] for k in range(X.shape[0])] for i in range(self.c)]
        
    def calculerV(self, X):
        card = [ sum([pow(self.U[i][k], self.m) for k in range(X.shape[0])]) for i in range(self.c)]
        term = [ [ sum([ pow(self.U[i][k], self.m)*X[k][j] for k in range(X.shape[0])]) for j in range(self.p)] for i in range(self.c)]
        self.V = [ [ term[i][j]/card[i] for j in range(self.p)] for i in range(self.c)]
    
    def classe(self,vect): 
        term1 = [ pow(sum(np.square(vect-self.V[i])), -1./(self.m-1)) for i in range(self.c)]
        term2 = sum([ pow(sum(np.square(vect-self.V[i])), -1./(self.m-1)) for i in range(self.c)])
        u = [term1[i]/term2 for i in range(self.c)]
        return np.argmax(u)
    

    def accuracy(self,X,Y):
        T = 0
        for i in range(len(X)):
            if self.classe(X[i]) == Y[i]:
                T+=1
        return T/len(X)
    
    def entrainer(self, X,Y): 
        t = 0
        while(t<self.m_iter):
            self.calculerU(X)
            self.calculerV(X)
            t = t + 1



In [17]:
fcm = FCM(X_train,2,2,100)
print(fcm.V.shape)

fcm.entrainer(np.array(X_train),y_train)


(2, 2304)
